In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class NN:
    def __init__(self):
        self.layersOfWeights = [] #Слои нейронов
        self.layersOfBiases = [] #Слои смещений
        self.activFunc = [] #Слои функций активации (Мало ли они будут разные)
        self.gradActivFunc = [] #Производная от слоя активации. Нужна на случай, если таковой нет(Тут будет просто функция, которая возвращает вход)
        self.countOfLayers = 0 #Просто число слоёв

        self.calculatedBeforeActivFunc = [] #Промежуточные значения до функции активации на каждом слое
        self.calculatedAfterActivFunc = [] #Промежуточные знчаения после вычисления фунцкции активации на каждом слое

        self.errorsDuringTraining = [] #Список ошибок на каждом шагу обучения

    def addLayer(self, n, m):
        """
        n - входное число нейронов<br>
        m - выходное число нейронов
        """
        q = 0.1
        self.layersOfWeights.append(np.random.uniform(-q, q, (n,m)))
        self.layersOfBiases.append(np.random.uniform(-q, q, (1,m)))
        self.countOfLayers += 1

    def addActivFunc(self, n:int=0):
        """
        n - номер функции активации: <br>
        <b>0</b> - никакой функции активации. Выбор по умолчанию<br>
        <b>1</b> - сигмойда (1/1 + exp(-x))<br>
        <b>2</b> - гиперболический тангенс (tanh(x))<br>
        <b>3</b> - softmax (exp(xi)) / sum(exp(xi))
        """
        none = lambda x: x
        gradNone = lambda x: np.ones_like(x)
        sigmoid = lambda x: 1 / (1 + np.exp(-x))
        gradSigmoid = lambda x: sigmoid(x) * (1 - sigmoid(x))
        tanh = lambda x: np.tanh(x)
        gradTanh = lambda x: 1 - (np.tanh(x))**2

        def softmax(x):
            # shift = x - np.max(x, axis=1, keepdims=True)
            # ex = np.exp(shift)
            # return ex / np.sum(ex, axis=1, keepdims=True)
        
            shift = x - np.max(x, axis=1, keepdims=True)
            shift = np.clip(shift, -20, +20)
            ex = np.exp(shift)
            return ex / np.sum(ex, axis=1, keepdims=True)

        def gradSoftmax(b:np.ndarray):
            """Градиент из softmax'a. <br>
            На вход принимает матрицу значений softmax, где строки - это один набор данных, а столбцы - значения выходных нейронов"""
            z = -b[:, :, None] * b[:, None]
            diag = np.diag_indices_from(z[0])
            z[:, diag[0], diag[1]] = b[:] * (1 - b[:])
            return z.sum(axis=1)

        if n ==0:
            func = none
            gradFunc = gradNone
        elif n ==1:
            func = sigmoid
            gradFunc = gradSigmoid
        elif n == 2: 
            func = tanh
            gradFunc = gradTanh
        elif n == 3:
            func = softmax
            gradFunc = gradSoftmax
        else:
            raise Exception("Выбрана неверная функция активации")
        
        self.activFunc.append(func)
        self.gradActivFunc.append(gradFunc)

    def predict(self, data:np.ndarray)->np.ndarray:
        """
        data - данные, на основе которых мы хотим получить предсказание
        """
        self.calculatedAfterActivFunc.append(data)
        a = data @ self.layersOfWeights[0] + self.layersOfBiases[0] #Первый слой
        self.calculatedBeforeActivFunc.append(a)
        a = self.activFunc[0](a)
        self.calculatedAfterActivFunc.append(a)
        
        for i in range(1, self.countOfLayers): #Все последующие слои, начиная со второго
            a = a @ self.layersOfWeights[i] + self.layersOfBiases[i]
            self.calculatedBeforeActivFunc.append(a)
            a = self.activFunc[i](a)
            self.calculatedAfterActivFunc.append(a)

        return a
    
    def _predict(self, data:np.ndarray)->np.ndarray:
        """
        <h3>Этот метод нужен для подсчёта внутри процесса обучения, что бы не влиять на него</h3>
        data - данные, на основе которых мы хотим получить предсказание
        """
        a = data @ self.layersOfWeights[0] + self.layersOfBiases[0] #Первый слой
        a = self.activFunc[0](a)
        if self.countOfLayers != 1:
            for i in range(1, self.countOfLayers): #Все последующие слои, начиная со второго
                a = a @ self.layersOfWeights[i] + self.layersOfBiases[i]
                a = self.activFunc[i](a)
        return a
    
    def _backProp(self, data:np.ndarray, true:np.ndarray, lr):
        """
        Метод обучения. Принимает в себя <b>(потенциально один)</b> экземпляр, на котором и учится 
        """
        self.calculatedBeforeActivFunc.clear()
        self.calculatedAfterActivFunc.clear()
        pred = self.predict(data) # Модель не обучалась потому что у тебя в предикте не заполнялись данные с прямого прохода
        
        e = ((pred-true)**2).mean()
        self.errorsDuringTraining.append(e)
        
        dedy = 2*(pred - true) / (pred.shape[0] * pred.shape[1]) # Тут ты видимо просто закомментил вычисление градиента
        self.dedy.append(dedy)

        #Первый слой
        dydz = dedy * self.gradActivFunc[-1](self.calculatedBeforeActivFunc[-1])
        self.dydz.append(dydz)

        dzdw = self.calculatedAfterActivFunc[-2].T @ dydz 
        self.dzdw.append(dzdw)

        dzdb = (dydz * 1).mean(axis=0)
        self.dzdb.append(dzdb)

        dzdy = dydz @ self.layersOfWeights[-1].T #updated
        self.dzdy.append(dzdy)

        self.layersOfWeights[-1] = self.layersOfWeights[-1] - dzdw * lr
        self.layersOfBiases[-1] = self.layersOfBiases[-1] - dzdb * lr

        #Все последующие слои, начиная со второго
        for i in range(1, self.countOfLayers):
            dydz = dzdy * self.gradActivFunc[-i-1](self.calculatedBeforeActivFunc[-i-1])
            dzdw = self.calculatedAfterActivFunc[-i-2].T @ dydz
            dzdb = (dydz * 1).mean(axis=0)
            self.layersOfBiases[-i-1] = self.layersOfBiases[-i-1] - dzdb * lr
            dzdy = dydz @ self.layersOfWeights[-i-1].T
            self.layersOfWeights[-i-1] = self.layersOfWeights[-i-1] - dzdw * lr #updated


    def training(self, data:np.ndarray, true:np.ndarray, steps, lr = 0.001, stopCreteria = 0.001, chart = False):
        self.errorsDuringTraining.clear()

        self.dedy = []
        self.dydz = []
        self.dzdw = []
        self.dzdb = []
        self.dzdy = []

        for i in range(steps):
            self._backProp(data, true, lr)
            # # Тут я даже не разбирался, ошибка в логике постоянно прерывала обучение.
            # if len(self.errorsDuringTraining) >= 2:
            #     if (self.errorsDuringTraining[-2] - self.errorsDuringTraining[-1]) < stopCreteria:
            #         print(f'Всего за {i} шагов модель обучилась достаточно')
            #         break
        if chart:
            plt.figure()
            plt.title("График обучения")
            plt.plot(range(self.errorsDuringTraining.__len__()), self.errorsDuringTraining)
            plt.xlabel("Номер итерации")
            plt.ylabel("Ошибка")

In [ ]:
"""Определение модели"""

net = NN()
# Входной слой: 4 признака → 10 нейронов, скрытый слой → 3 нейрона на выходе
net.addLayer(4, 10)
net.addActivFunc(2)   # tanh
net.addLayer(10, 3)
net.addActivFunc(3)   # softmax

In [ ]:
"""Загружаем датасет"""

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# 1. Загружаем датасет Iris
iris = datasets.load_iris()
X = iris.data               # shape (150, 4)
y = iris.target.reshape(-1, 1)  # shape (150, 1)

# 2. One‑hot кодирование меток (3 класса)
encoder = OneHotEncoder(sparse_output=False)
Y = encoder.fit_transform(y)  # shape (150, 3)

# 3. Разбиваем на тренировочную и тестовую выборки
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42, stratify=Y
)

# 4. Нормализуем вход
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform (X_test)

# 6. Оцениваем точность до обучения
pred_probs = net.predict(X_test)
pred_labels_nt = np.argmax(pred_probs, axis=1)
true_labels = np.argmax(Y_test, axis=1)
accuracy = (pred_labels_nt == true_labels).mean()
print(f"Start accuracy: {accuracy:.3f}")

In [ ]:
# 5. Обучаем
net.training(X_train, Y_train,
             steps=20000,
             lr=0.001,
             stopCreteria=1e-5,
             chart=True)

In [ ]:

# 6. Оцениваем точность на тесте
pred_probs = net.predict(X_test)
pred_labels = np.argmax(pred_probs, axis=1)
true_labels = np.argmax(Y_test, axis=1)
accuracy = (pred_labels == true_labels).mean()
print(f"Test accuracy: {accuracy:.3f}")

In [ ]:
# Смотрим как изменились метки после обучения

import pandas as pd

# 5) Собираем DataFrame
df_results = pd.DataFrame({
    'TrueLabel':   true_labels,
    'PredBefore':  pred_labels_nt,
    'PredAfter':   pred_labels
})

df_results['Changed?'] = df_results['PredBefore'] != df_results['PredAfter']

df_results


In [ ]:
import itertools

# число прогонов для каждой комбинации (чем больше, тем точнее, но дольше)
n_repeats = 5

# 2) Сетка гиперпараметров
layer_configs = [
    [10],        # одна скрытая из 10 нейронов
    [20],        # одна скрытая из 20
    [10, 10],    # две по 10
    [3, 5, 3],
]
activ_choices = {
    1: "sigmoid",
    2: "tanh",
    3: "softmax"
}
weight_scales = [0.1, 1.0, 0.01]
lrs = [0.001, 0.01, 0.1]

results = []

# 3) Grid search с повторениями
for hidden, act_code, scale, lr in itertools.product(
        layer_configs, activ_choices, weight_scales, lrs):

    accs = []
    for run in range(n_repeats):
        # фиксируем семя, чтобы разные запуски отличались, но было воспроизводимо
        seed = 42 + run
        np.random.seed(seed)

        net = NN()
        prev_dim = X_train.shape[1]
        # создаём скрытые слои
        for size in hidden:
            net.addLayer(prev_dim, size)
            net.addActivFunc(act_code)
            prev_dim = size
        # выходной softmax-слой
        net.addLayer(prev_dim, Y_train.shape[1])
        net.addActivFunc(3)
        # масштабируем стартовые веса
        for i in range(len(net.layersOfWeights)):
            net.layersOfWeights[i] *= scale

        # обучаем
        net.training(
            X_train, Y_train,
            steps=200,
            lr=lr,
            chart=False
        )

        # валидация
        pred = net.predict(X_test)
        pred_labels = np.argmax(pred, axis=1)
        true_labels = np.argmax(Y_test, axis=1)
        accs.append((pred_labels == true_labels).mean())

    # после n_repeats считаем метрики стабильности
    mean_acc = np.mean(accs)
    std_acc  = np.std(accs)

    results.append({
        "hidden_layers": hidden,
        "activation":   activ_choices[act_code],
        "weight_scale": scale,
        "learning_rate": lr,
        "mean_acc":     mean_acc,
        "std_acc":      std_acc,
        "all_runs":     accs
    })

# 4) Сводная таблица: сортируем по убыванию mean_acc, затем по возрастанию std_acc
df = pd.DataFrame(results)
df = df.sort_values(["mean_acc", "std_acc"], ascending=[False, True]).reset_index(drop=True)

# посмотрим топ-10 самых стабильных и высоко среднеработающих конфигураций
df.head(10)
